##### *Copyright 2021 Google LLC*
*Licensed under the Apache License, Version 2.0 (the "License")*

In [ ]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Train EfficientDet for Object Detection with TensorFlow Lite Model Maker

## Import the required packages

In [ ]:
!pip install -q tflite-model-maker

     |████████████████████████████████| 642 kB 5.1 MB/s 
     |████████████████████████████████| 42.5 MB 199 kB/s 
     |████████████████████████████████| 6.4 MB 54.7 MB/s 
     |████████████████████████████████| 237 kB 68.7 MB/s 
     |████████████████████████████████| 3.4 MB 54.1 MB/s 
     |████████████████████████████████| 840 kB 62.0 MB/s 
     |████████████████████████████████| 1.1 MB 60.9 MB/s 
     |████████████████████████████████| 1.2 MB 49.5 MB/s 
     |████████████████████████████████| 11.2 MB 53.5 MB/s 
     |████████████████████████████████| 87 kB 6.7 MB/s 
     |████████████████████████████████| 77 kB 5.5 MB/s 
     |████████████████████████████████| 596 kB 20.4 MB/s 
     |████████████████████████████████| 120 kB 62.9 MB/s 
     |████████████████████████████████| 25.3 MB 66.4 MB/s 
     |████████████████████████████████| 352 kB 64.5 MB/s 
     |████████████████████████████████| 48.3 MB 188 kB/s 
     |████████████████████████████████| 99 kB 9.2 MB/s 
     |█████████████

In [ ]:
import numpy as np
import os

from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector

import tensorflow as tf
assert tf.__version__.startswith('2')

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)


In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following packages will be REMOVED:
  libcudnn8-dev
The following held packages will be changed:
  libcudnn8
The following packages will be upgraded:
  libcudnn8
1 upgraded, 0 newly installed, 1 to remove and 43 not upgraded.
Need to get 430 MB of archives.
After this operation, 3,139 MB disk space will be freed.
Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  libcudnn8 8.1.0.77-1+cuda11.2 [430 MB]
Fetched 430 MB in 8s (52.1 MB/s)
(Reading database ... 155632 files and directories currently installed.)
Removing libcudnn8-dev (8.0.5.39-1+cuda11.1) ...
(Reading database ... 155610 files and directories currently installed.)
Preparing to unpack .../libcudnn8_8.1.0.77-1+cuda11.2_amd64.deb ...
Unpacking libcudnn8 (8.1.0.77-1+c

## Load the training data


To use the default salad training dataset, just run all the code below as-is.

But if you want to train with your own image dataset, follow these steps:

1. Be sure your dataset is annotated in Pascal VOC XML (various tools can help create VOC annotations, such as [LabelImg](https://github.com/tzutalin/labelImg#labelimg)). Then create a ZIP file with all your JPG images and XML files (JPG and XML files can all be in one directory or in separate directories).
2. Click the **Files** tab in the left panel and just drag-drop your ZIP file there to upload it.
3. Use the following drop-down option to set **`use_custom_dataset`** to True.
4. If your dataset is already split into separate directories for training, validation, and testing, also set **`dataset_is_split`** to True. (If your dataset is not split, leave it False and we'll split it below.)
5. Then skip to [Load your own Pascal VOC dataset](#scrollTo=ZljJ25RAnj5x) and follow the rest of the instructions there.




###Load the Pascal VOC dataset

To use your custom dataset, you need to modify a few variables here, such as your ZIP filename, your label map, and the path to your images/annotations:

In [ ]:
use_custom_dataset = True
dataset_is_split = True 

In [ ]:
if use_custom_dataset:

  # # The ZIP file you uploaded:
  !unzip datasetmerge.zip

  # Your labels map as a dictionary (zero is reserved):
  label_map = {1: 'Jerawat', 2: 'Komedo'} 

  if dataset_is_split:
    # If your dataset is already split, specify each path:
    train_images_dir = 'dataset (1)/train/images'
    train_annotations_dir = 'dataset (1)/train/annotations'
    val_images_dir = 'dataset (1)/valid/images'
    val_annotations_dir = 'dataset (1)/valid/annotations'
    test_images_dir = 'dataset (1)/test/images'
    test_annotations_dir = 'dataset (1)/test/annotations'
  else:
    # If it's NOT split yet, specify the path to all images and annotations
    images_in = 'dataset (1)/images'
    annotations_in = 'dataset (1)/annotations'
  

Output streaming akan dipotong hingga 5000 baris terakhir.
  inflating: dataset (1)/train/images/acne-open-comedo-19_jpg.rf.91e6802063fb75c0e3d842cd336f51dd.jpg  
  inflating: dataset (1)/train/images/acne-open-comedo-19_jpg.rf.dc3f4d6e5912191b3c413ce93f327c4e.jpg  
  inflating: dataset (1)/train/images/acne-open-comedo-2_jpg.rf.152c709d5654eb1781f9c8645bd4de67.jpg  
  inflating: dataset (1)/train/images/acne-open-comedo-2_jpg.rf.57c93a71035a3c069405a229964b66dc.jpg  
  inflating: dataset (1)/train/images/acne-open-comedo-2_jpg.rf.9c7e56d0e93012d27bb04c61cc31308a.jpg  
  inflating: dataset (1)/train/images/acne-open-comedo-2_jpg.rf.a405763f6ad7e3a96c4d96d24ba1d8a9.jpg  
  inflating: dataset (1)/train/images/acne-open-comedo-2_jpg.rf.c56e3336fa1e08369157154b04f67877.jpg  
  inflating: dataset (1)/train/images/acne-open-comedo-2_jpg.rf.ef0e4e1bf35fcd6422c9e5eac38f304e.jpg  
  inflating: dataset (1)/train/images/acne-open-comedo-2_jpg.rf.fb09b99e04c3e08e3af6e8ac31072c89.jpg  
  inflating:

In [ ]:
import os
import random
import shutil

def split_dataset(images_path, annotations_path, val_split, test_split, out_path):
  """Splits a directory of sorted images/annotations into training, validation, and test sets.

  Args:
    images_path: Path to the directory with your images (JPGs).
    annotations_path: Path to a directory with your VOC XML annotation files,
      with filenames corresponding to image filenames. This may be the same path
      used for images_path.
    val_split: Fraction of data to reserve for validation (float between 0 and 1).
    test_split: Fraction of data to reserve for test (float between 0 and 1).
  Returns:
    The paths for the split images/annotations (train_dir, val_dir, test_dir)
  """
  _, dirs, _ = next(os.walk(images_path))

  train_dir = os.path.join(out_path, 'train')
  val_dir = os.path.join(out_path, 'validation')
  test_dir = os.path.join(out_path, 'test')

  IMAGES_TRAIN_DIR = os.path.join(train_dir, 'images')
  IMAGES_VAL_DIR = os.path.join(val_dir, 'images')
  IMAGES_TEST_DIR = os.path.join(test_dir, 'images')
  os.makedirs(IMAGES_TRAIN_DIR, exist_ok=True)
  os.makedirs(IMAGES_VAL_DIR, exist_ok=True)
  os.makedirs(IMAGES_TEST_DIR, exist_ok=True)

  ANNOT_TRAIN_DIR = os.path.join(train_dir, 'annotations')
  ANNOT_VAL_DIR = os.path.join(val_dir, 'annotations')
  ANNOT_TEST_DIR = os.path.join(test_dir, 'annotations')
  os.makedirs(ANNOT_TRAIN_DIR, exist_ok=True)
  os.makedirs(ANNOT_VAL_DIR, exist_ok=True)
  os.makedirs(ANNOT_TEST_DIR, exist_ok=True)

  # Get all filenames for this dir, filtered by filetype
  filenames = os.listdir(os.path.join(images_path))
  filenames = [os.path.join(images_path, f) for f in filenames if (f.endswith('.jpg'))]
  # Shuffle the files, deterministically
  filenames.sort()
  random.seed(42)
  random.shuffle(filenames)
  # Get exact number of images for validation and test; the rest is for training
  val_count = int(len(filenames) * val_split)
  test_count = int(len(filenames) * test_split)
  for i, file in enumerate(filenames):
    source_dir, filename = os.path.split(file)
    annot_file = os.path.join(annotations_path, filename.replace("jpg", "xml"))
    if i < val_count:
      shutil.copy(file, IMAGES_VAL_DIR)
      shutil.copy(annot_file, ANNOT_VAL_DIR)
    elif i < val_count + test_count:
      shutil.copy(file, IMAGES_TEST_DIR)
      shutil.copy(annot_file, ANNOT_TEST_DIR)
    else:
      shutil.copy(file, IMAGES_TRAIN_DIR)
      shutil.copy(annot_file, ANNOT_TRAIN_DIR)
  return (train_dir, val_dir, test_dir)

In [ ]:
# We need to instantiate a separate DataLoader for each split dataset
if use_custom_dataset:
  if dataset_is_split:
    train_data = object_detector.DataLoader.from_pascal_voc(
        train_images_dir, train_annotations_dir, label_map=label_map)
    validation_data = object_detector.DataLoader.from_pascal_voc(
        val_images_dir, val_annotations_dir, label_map=label_map)
    test_data = object_detector.DataLoader.from_pascal_voc(
        test_images_dir, test_annotations_dir, label_map=label_map)
  else:
    train_dir, val_dir, test_dir = split_dataset(images_in, annotations_in,
                                                 val_split=0.2, test_split=0.2,
                                                 out_path='split-dataset')
    train_data = object_detector.DataLoader.from_pascal_voc(
        os.path.join(train_dir, 'images'),
        os.path.join(train_dir, 'annotations'), label_map=label_map)
    validation_data = object_detector.DataLoader.from_pascal_voc(
        os.path.join(val_dir, 'images'),
        os.path.join(val_dir, 'annotations'), label_map=label_map)
    test_data = object_detector.DataLoader.from_pascal_voc(
        os.path.join(test_dir, 'images'),
        os.path.join(test_dir, 'annotations'), label_map=label_map)
    
  print(f'train count: {len(train_data)}')
  print(f'validation count: {len(validation_data)}')
  print(f'test count: {len(test_data)}')

train count: 5182
validation count: 449
test count: 455


## Select the model spec

Model Maker supports the EfficientDet-Lite family of object detection models that are compatible with the Edge TPU. (EfficientDet-Lite is derived from [EfficientDet](https://ai.googleblog.com/2020/04/efficientdet-towards-scalable-and.html), which offers state-of-the-art accuracy in a small model size). There are several model sizes you can choose from:

|| Model architecture | Size(MB)* | Latency(ms)** | Average Precision*** |
|-|--------------------|-----------|---------------|----------------------|
|| EfficientDet-Lite0 | 5.7       | 37.4            | 30.4%               |
|| EfficientDet-Lite1 | 7.6       | 56.3            | 34.3%               |
|| EfficientDet-Lite2 | 10.2      | 104.6           | 36.0%               |
|| EfficientDet-Lite3 | 14.4      | 107.6           | 39.4%               |
| <td colspan=4><br><i>* File size of the compiled Edge TPU models. <br/>** Latency measured on a desktop CPU with a Coral USB Accelerator. <br/>*** Average Precision is the mAP (mean Average Precision) on the COCO 2017 validation dataset.</i></td> |


In [ ]:
spec = object_detector.EfficientDetLite2Spec()

## Create and train the model

In [ ]:
model = object_detector.create(train_data=train_data, 
                               model_spec=spec, 
                               validation_data=validation_data, 
                               epochs=100, 
                               batch_size=8, 
                               train_whole_model=True)

Epoch 1/100
647/647 [==============================] - 306s 391ms/step - det_loss: 1.3462 - cls_loss: 0.8203 - box_loss: 0.0105 - reg_l2_loss: 0.0761 - loss: 1.4223 - learning_rate: 0.0090 - gradient_norm: 2.6267 - val_det_loss: 1.3004 - val_cls_loss: 0.8033 - val_box_loss: 0.0099 - val_reg_l2_loss: 0.0763 - val_loss: 1.3767
Epoch 2/100
647/647 [==============================] - 249s 384ms/step - det_loss: 1.1721 - cls_loss: 0.7171 - box_loss: 0.0091 - reg_l2_loss: 0.0765 - loss: 1.2485 - learning_rate: 0.0100 - gradient_norm: 2.2739 - val_det_loss: 1.1319 - val_cls_loss: 0.6486 - val_box_loss: 0.0097 - val_reg_l2_loss: 0.0766 - val_loss: 1.2085
Epoch 3/100
647/647 [==============================] - 249s 384ms/step - det_loss: 1.1280 - cls_loss: 0.6888 - box_loss: 0.0088 - reg_l2_loss: 0.0767 - loss: 1.2046 - learning_rate: 0.0100 - gradient_norm: 2.1082 - val_det_loss: 1.1686 - val_cls_loss: 0.6863 - val_box_loss: 0.0096 - val_reg_l2_loss: 0.0767 - val_loss: 1.2453
Epoch 4/100
647/647

## Evaluate the model

In [ ]:
model.evaluate(test_data,batch_size=8)

57/57 [==============================] - 17s 207ms/step



{'AP': 0.053873956,
 'AP50': 0.18882218,
 'AP75': 0.017440021,
 'AP_/Jerawat': 0.076047204,
 'AP_/Komedo': 0.031700708,
 'APl': 0.09532971,
 'APm': 0.07889529,
 'APs': 0.03837634,
 'ARl': 0.3377907,
 'ARm': 0.2874344,
 'ARmax1': 0.05719419,
 'ARmax10': 0.16751757,
 'ARmax100': 0.2456929,
 'ARs': 0.21541898}

## Export to TensorFlow Lite

In [ ]:
TFLITE_FILENAME = 'camerlang-efficientdet-lite3.tflite'
LABELS_FILENAME = 'camerlang-labels.txt'
SAVED_MODEL_FILENAME = 'camerlang-saved-model.pb'

In [ ]:
model.export(export_dir='.', 
             tflite_filename=TFLITE_FILENAME,
             label_filename=LABELS_FILENAME,
             saved_model_filename=SAVED_MODEL_FILENAME,
             export_format=[ExportFormat.TFLITE, ExportFormat.LABEL, ExportFormat.SAVED_MODEL])

### Evaluate the TF Lite model

In [ ]:
model.evaluate_tflite(TFLITE_FILENAME, 
                      test_data)

455/455 [==============================] - 237s 519ms/step



{'AP': 0.05073047,
 'AP50': 0.17533718,
 'AP75': 0.01669018,
 'AP_/Jerawat': 0.07448943,
 'AP_/Komedo': 0.02697151,
 'APl': 0.102413885,
 'APm': 0.074214354,
 'APs': 0.035588466,
 'ARl': 0.3230378,
 'ARm': 0.23718141,
 'ARmax1': 0.05300056,
 'ARmax10': 0.15717961,
 'ARmax100': 0.18588161,
 'ARs': 0.14771584}

### Test the TFLite model

In [ ]:
import random

# If you're using a custom dataset, we take a random image from the test set:
if use_custom_dataset:
  images_path = test_images_dir if dataset_is_split else os.path.join(test_dir, "images")
  filenames = os.listdir(os.path.join(images_path))
  random_index = random.randint(0,len(filenames)-1)
  INPUT_IMAGE = os.path.join(images_path, filenames[random_index])
else:
  # Download a test jerawat image from browser
  INPUT_IMAGE = 'jerawat-test.jpg'
  DOWNLOAD_URL = "https://www.herworld.co.id/gallery/teaser/penyebab-jerawat-di-pipi1_6720220217122411sx8LGK.jpg"
  !wget -q -O $INPUT_IMAGE $DOWNLOAD_URL

To simplify our code, we'll use the [PyCoral API](https://coral.ai/docs/reference/py/):

In [ ]:
! python3 -m pip install --extra-index-url https://google-coral.github.io/py-repo/ pycoral

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://google-coral.github.io/py-repo/
     |████████████████████████████████| 373 kB 5.2 MB/s 
     |████████████████████████████████| 1.5 MB 921 kB/s 


In [ ]:
from PIL import Image
from PIL import ImageDraw
from PIL import ImageFont

import tflite_runtime.interpreter as tflite 
from pycoral.adapters import common
from pycoral.adapters import detect
from pycoral.utils.dataset import read_label_file

def draw_objects(draw, objs, scale_factor, labels):
  """Draws the bounding box and label for each object."""
  COLORS = np.random.randint(0, 255, size=(len(labels), 3), dtype=np.uint8)
  for obj in objs:
    bbox = obj.bbox
    color = tuple(int(c) for c in COLORS[obj.id])
    draw.rectangle([(bbox.xmin * scale_factor, bbox.ymin * scale_factor),
                    (bbox.xmax * scale_factor, bbox.ymax * scale_factor)],
                   outline=color, width=3)
    font = ImageFont.truetype("LiberationSans-Regular.ttf", size=15)
    draw.text((bbox.xmin * scale_factor + 4, bbox.ymin * scale_factor + 4),
              '%s\n%.2f' % (labels.get(obj.id, obj.id), obj.score),
              fill=color, font=font)

# Load the TF Lite model
labels = read_label_file(LABELS_FILENAME)
interpreter = tflite.Interpreter(TFLITE_FILENAME)
interpreter.allocate_tensors()

# Resize the image for input
image = Image.open(INPUT_IMAGE)
_, scale = common.set_resized_input(
    interpreter, image.size, lambda size: image.resize(size, Image.ANTIALIAS))

# Run inference
interpreter.invoke()
objs = detect.get_objects(interpreter, score_threshold=0.3, image_scale=scale)

# Resize again to a reasonable size for display
display_width = 500
scale_factor = display_width / image.width
height_ratio = image.height / image.width
image = image.resize((display_width, int(display_width * height_ratio)))
draw_objects(ImageDraw.Draw(image), objs, scale_factor, labels)
image

ImportError: ignored

## Download the files

In [ ]:
from google.colab import files

files.download(TFLITE_FILENAME)
files.download(TFLITE_FILENAME.replace('.tflite', '_edgetpu.tflite'))
files.download(LABELS_FILENAME)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

FileNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
